In [1]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Get current directory
CURR_DIR = os.getcwd()

# Get MNIST 
# One-hot encoding is used for categorical variables, as binary vector
mnist = input_data.read_data_sets(CURR_DIR + "/data/", one_hot=True)

Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/13. TensorFlow_Playground/data/train-images-idx3-ubyte.gz
Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/13. TensorFlow_Playground/data/train-labels-idx1-ubyte.gz
Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/13. TensorFlow_Playground/data/t10k-images-idx3-ubyte.gz
Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/13. TensorFlow_Playground/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.1
num_steps = 1000
batch_size = 128
display_step = 100

In [4]:
# Network Parameters
n_hidden_1 = 256
n_hidden_2 = 256
num_input = 784
num_classes = 10

In [5]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)

In [26]:
# Define the neural network
def neural_net(x_dict):
    # TF Estimator input is a dict, in case of multiple inputs
    x = x_dict['images']
    
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(x, n_hidden_1)
    
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2)
    
    # Output fully connected layer with a neuron for each class
    out_layer = tf.layers.dense(layer_2, num_classes)
    return out_layer

In [33]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    logits = neural_net(features)
    
    # Predictions
    pred_classes = tf.argmax(logits, axis=1)
    pred_probas = tf.nn.softmax(logits)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [34]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_session_config': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/tr/b_lmltt51gg6_65v22ydd5fh0000gn/T/tmpov93cvn2', '_keep_checkpoint_max': 5}


In [35]:
# Train the Model
model.train(input_fn, steps=num_steps)

ValueError: None values not supported.

In [30]:
# Evaluate the Model
# Define the input function for evaluating
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
# Use the Estimator 'evaluate' method
model.evaluate(input_fn)

ValueError: Could not find trained model in model_dir: /var/folders/tr/b_lmltt51gg6_65v22ydd5fh0000gn/T/tmp7xc5i9wu.

In [31]:
# Predict single images
n_images = 4
# Get images from test set
test_images = mnist.test.images[:n_images]
# Prepare the input data
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': test_images}, shuffle=False)
# Use the model to predict the images class
preds = list(model.predict(input_fn))

ValueError: Could not find trained model in model_dir: /var/folders/tr/b_lmltt51gg6_65v22ydd5fh0000gn/T/tmp7xc5i9wu.

In [32]:
# Display
for i in range(n_images):
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')
    plt.show()
    print("Model prediction:", preds[i])

NameError: name 'plt' is not defined